# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_data = pd.read_csv("output_data/cities.csv")
weather_data


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Tuktoyaktuk,90,CA,1620236737,86,69.4541,-133.0374,19.40,3.44
1,Adrar,2,MR,1620236738,14,20.5022,-10.0711,89.94,13.20
2,Española,1,US,1620236446,25,35.9911,-106.0806,70.00,6.91
3,Aksu,6,CN,1620236738,15,41.1231,80.2644,69.24,5.08
4,Port Elizabeth,0,ZA,1620236738,94,-33.9180,25.5701,66.20,3.44
...,...,...,...,...,...,...,...,...,...
574,Erenhot,100,CN,1620236877,29,43.6475,111.9767,51.73,34.00
575,Kem',100,RU,1620236877,78,64.9555,34.5793,31.33,5.21
576,Naze,100,JP,1620236878,70,28.3667,129.4833,65.08,8.57
577,Taoudenni,74,ML,1620236878,19,22.6783,-3.9836,94.08,11.99


In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [6]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
max_intensity

100

In [8]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
97,Vicuña,0,CL,1620236760,31,-30.0319,-70.7081,70.00,6.20
99,Fayaoué,0,NC,1620236761,78,-20.6500,166.5333,75.88,5.08
309,Qiongshan,0,CN,1620236812,94,20.0058,110.3542,78.80,4.47
329,Ceyhan,0,TR,1620236816,94,37.0247,35.8175,71.60,6.91
400,Haifa,0,IL,1620236835,38,32.8156,34.9892,78.80,3.44
481,İmamoğlu,0,TR,1620236854,94,37.2651,35.6572,71.60,6.91


In [9]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
97,Vicuña,CL,-30.0319,-70.7081,
99,Fayaoué,NC,-20.6500,166.5333,
309,Qiongshan,CN,20.0058,110.3542,
329,Ceyhan,TR,37.0247,35.8175,
400,Haifa,IL,32.8156,34.9892,
481,İmamoğlu,TR,37.2651,35.6572,


In [10]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [16]:

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
   # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

 # Display the hotel dataframe
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
97,Vicuña,CL,-30.0319,-70.7081,
99,Fayaoué,NC,-20.6500,166.5333,
309,Qiongshan,CN,20.0058,110.3542,
329,Ceyhan,TR,37.0247,35.8175,
400,Haifa,IL,32.8156,34.9892,
481,İmamoğlu,TR,37.2651,35.6572,


In [19]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:

# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))